The first cells are merely used to extract data of our already made dataframes.

In [36]:
import pickle
import pandas as pd 

with open('dataframe.pkl', 'rb') as f:  # Open in binary mode
    df = pickle.load(f)  # Load the DataFrame

with open('dataframe_perturbated.pkl', 'rb') as f:  # Open in binary mode
    df2 = pickle.load(f)  # Load the DataFrame

with open('dataframe_perturbated_maths.pkl', 'rb') as f:  # Open in binary mode
    df3 = pickle.load(f)  # Load the DataFrame

In [42]:
# Calculate the average length of "Wuestion" column in df
avg_length_df = df["Question"].astype(str).apply(len).mean()

# Calculate the average length of "Wuestion" column in df2
avg_length_df2 = df2["Question"].astype(str).apply(len).mean()

# Calculate the average length of "Wuestion" column in df2
avg_length_df3 = df3["Question"].astype(str).apply(len).mean()

print(f"Average length of 'Question' in df: {avg_length_df:.2f}")
print(f"Average length of 'Question' in df2: {avg_length_df2:.2f}")
print(f"Average length of 'Question' in df2: {avg_length_df3:.2f}")

Average length of 'Question' in df: 191.27
Average length of 'Question' in df2: 262.58
Average length of 'Question' in df2: 262.58


In [53]:
# Assuming you already have a dataframe 'df'
for col in df2.columns:
    print(col)        # Prints the column name
    print(df2[col][29])    # Prints all values in that column
    break

# Assuming you already have a dataframe 'df'
for col in df.columns:
    print(col)        # Prints the column name
    print(df[col][135 + 29])    # Prints all values in that column
    break

Question
It takes 3 weeks to prepare for a camp. Coordination time is one-third of the preparation time plus 10% more of the preparation time. If the planning phase is twice the coordination time but minus 15% of the preparation time, what is the total duration of the event?
Question
For a camp, preparation takes 3 weeks. coordination is a third of preparation plus 10% of preparation. If planning is double the duration of coordination minus 15% of preparation, how long is the event?


In [ ]:
from openai import OpenAI
client = OpenAI(api_key='xxx')

In [7]:
import pickle
import json
import pandas as pd 

# Load the DataFrame from the pickle file
with open('dataframe.pkl', 'rb') as f:
    df = pickle.load(f)  

df = df.head(1000)

# Define a system message that remains constant across all requests.
system_message = "You are a perturbation assistant. Your task is to rephrase the given input following the provided instructions to generate a perturbed version of the original question."

# Create the JSONL file for batch processing
jsonl_file = "batch_requests.jsonl"
with open(jsonl_file, "w") as f:
    for index, row in df.iterrows():
        # Create the prompt with the necessary details
        prompt_text = f"""
You are provided with a single question from an LLM benchmark. The input consists of the following fields:

- **Question:** {row['Question']}
- **Option A:** {row['Option A']}
- **Option B:** {row['Option B']}
- **Option C:** {row['Option C']}
- **Answer:** {row['Answer']}
- **Category:** {row['Category']}

Your task is to generate a *perturbed version* of this input. To do this, follow these steps:

1. **Reference Input:** (Optionally, include a reference to the original input as a comment for internal processing, but do not include it in the final output.)
2. **Alter the Question:** Change the question using numbers while keeping its wording intact.
3. **Correct the Options:** Now correct the correct option accordingly, but ensure the mapping to the correct answer remains unchanged.
4. **Preserve Answer & Category:** The correct answer and the category must remain exactly as provided.
5. **Output Format:** Present the perturbed version in the same structured format as the input.
        """
        # Build the JSON object for this request following the sample formatting
        request_obj = {
            "custom_id": f"request-{index}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4-turbo", # -2024-12-17
                "messages": [
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": prompt_text.strip()}
                ],
                "max_tokens": 1000
            }
        }
        # Write each JSON object as a separate line in the JSONL file.
        f.write(json.dumps(request_obj) + "\n")

print(f"Batch JSONL file '{jsonl_file}' created with {len(df)} requests.")

Batch JSONL file 'batch_requests.jsonl' created with 1000 requests.


In [8]:
batch_input_file = client.files.create(
    file=open("batch_requests.jsonl", "rb"),
    purpose="batch"
)

batch_input_file_id = batch_input_file.id
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "nightly eval job"
    }
)

Batch(id='batch_67eab1cb406c8190b90c1e6d9edd146b', completion_window='24h', created_at=1743434187, endpoint='/v1/chat/completions', input_file_id='file-QnwUqu6SE4YzcVBHRrqLKi', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1743520587, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [10]:
batch = client.batches.retrieve("batch_67eab1cb406c8190b90c1e6d9edd146b")
print(batch)

Batch(id='batch_67eab1cb406c8190b90c1e6d9edd146b', completion_window='24h', created_at=1743434187, endpoint='/v1/chat/completions', input_file_id='file-QnwUqu6SE4YzcVBHRrqLKi', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1743434509, error_file_id=None, errors=None, expired_at=None, expires_at=1743520587, failed_at=None, finalizing_at=1743434437, in_progress_at=1743434188, metadata={'description': 'nightly eval job'}, output_file_id='file-HAgRNmcncDgburP6N8ykxV', request_counts=BatchRequestCounts(completed=1000, failed=0, total=1000))


Save bash output to jsonl file.

In [ ]:
file_response = client.files.content("file-R3Lbe3GZmvSmPkTJCznjX3")

with open('batch_maths_gpt4.jsonl', 'w') as f:
    f.write(file_response.text)